<a href="https://colab.research.google.com/github/josephthomaa/ML-Notebooks/blob/main/Llamaindex_with_opensource_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q llama-index
!pip install -q openai
!pip install -q transformers
!pip install -q accelerate
!pip install -q optimum[exporters]
!pip install -q InstructorEmbedding
!pip install -q sentence_transformers
!pip install -q pypdf
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
!pip install -q llama-index chromadb --quiet
!pip install -q chromadb
!pip install -q sentence-transformers
!pip install -q pydantic==1.10.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
# import
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import chromadb


In [4]:
# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [5]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 28.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.26-cp310-cp310-manylinux_2_35_x86_64.whl size=8129952 sha256=c2d1084688d572d864c78b44ed6f2b2c3a3c8944971e98fb3e79563f60aef80e
  Stored in directory: /tmp/pip-ephem-wheel-cache-m84cuj93/wheels/91/80/ce/ac6afea8c1d6fbcec7e14183033a5b2796c742d4f470010c72
Successfully built llama-cpp-python


In [6]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


total size (MB): 4368.44


4167it [00:21, 190.38it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
# !curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0   252k      0  0:01:24  0:01:24 --:--:--  265k


In [ ]:
documents = SimpleDirectoryReader(
    input_files=["IPCC_AR6_WGII_Chapter03.pdf"]
).load_data()

In [ ]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("orca_paper")

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [ ]:

index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context,
                                        service_context=service_context)

In [ ]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("Which livelihoods and economic sectors are most vulnerable to the impacts of climate change in the oceans?")

display(Markdown(f"{response}"))

Llama.generate: prefix-match hit


 The livelihoods and economic sectors that are most vulnerable to the impacts of climate change in the oceans include small-scale fisheries, tourism, real estate values, and coastal activities. These industries and jobs depend on suitable environmental conditions and are at risk from changes in marine resources and ecosystem services due to climate change.

### Saving to Disk

In [7]:
! pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=153873fc724598ca9703fab0e7953d6e49a2de2a3f0bb6c23fc66c22995b5ff1
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [8]:
documents = SimpleDirectoryReader("data/").load_data()

In [9]:
db = chromadb.PersistentClient(path="./chroma_db")


chroma_collection = db.get_or_create_collection("DB_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

service_context = ServiceContext.from_defaults(embed_model=embed_model,
                                               chunk_size=1200,
                                               chunk_overlap=100,
                                               llm = llm)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

### Load from Disk

In [10]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")

chroma_collection = db2.get_or_create_collection("DB_collection")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [11]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("How should the Amazon Operations Team prepare for engaging the Amazon Development Team?")

display(Markdown(f"{response}"))

 To prepare for engaging the Amazon Development Team, the Amazon Operations Team should follow these steps:

1. Ensure access to the Jira platform and designated Slack channels for incident triage.
2. Maintain an up-to-date list of Amazon Development Team members who should be notified in case of production issues.
3. Identify the nature and severity of the production issue requiring the involvement of the Amazon Development Team.
4. Log in to Jira using your credentials and initiate the process to create a new Jira issue for the incident.
5. Assign the Jira ticket to a member of the Amazon Development Team responsible for incident resolution and notify relevant stakeholders about the new Jira ticket.
6. Utilize the designated Slack channel (#incident-triage) for incident triage discussions and share the generated Jira ticket link within the message to provide context and access to the incident details.
7. Engage Amazon Development Team members and relevant stakeholders in discussions within the Slack channel, analyzing the Jira ticket details, impact, potential causes, and initial steps taken.
8. If critical issues require immediate attention, use the provided emergency contact information for escalation.
9

In [ ]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("List all actions to be done by operation team when there is a low order volume . Also mention if there is any automation tasks ops team can use with low order")

display(Markdown(f"{response}"))

### Viewing/Customizing Prompts

In [12]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [13]:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

### Custom prompt

In [17]:
! pip install -q langchain
! pip install -q langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kubernetes 28.1.0 requires urllib3<2.0,>=1.24.2, but you have urllib3 2.1.0 which is incompatible.


In [18]:
from langchain import hub

langchain_prompt = hub.pull("rlm/rag-prompt")

In [19]:
from llama_index.prompts import LangchainPromptTemplate

lc_prompt_tmpl = LangchainPromptTemplate(
    template=langchain_prompt,
    template_var_mappings={"query_str": "question", "context_str": "context"},
)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": lc_prompt_tmpl}
)

prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [20]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What is the objective of the \"Engaging Amazon Dev Team on Production Issue\" SOP?")

display(Markdown(f"{response}"))

Llama.generate: prefix-match hit


 The objective of the "Engaging Amazon Dev Team on Production Issue" Standard Operating Procedure (SOP) is to outline the steps for the Amazon Operations Team to engage the Amazon Development Team when a production issue arises. This includes creating a dummy Jira ticket for the incident, utilizing a designated Slack channel for incident triage, and initiating incident triage discussions with the Jira ticket link.

In [31]:

qa_prompt_tmpl_str = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, \
answer the query below. Give answer in bullet points if applicable . If you don't know the answer,\
just say that you don't know, don't try to make up an answer.

Query: {query_str}
Answer: \
"""

In [32]:
from llama_index.prompts import PromptTemplate

qa_prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str,
    # function_mappings={"few_shot_examples": few_shot_examples_fn},
)

In [33]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

display_prompt_dict(query_engine.get_prompts())

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query below. Give answer in bullet points if applicable . If you don't know the answer,just say that you don't know, don't try to make up an answer.

Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [35]:
# Query Data
query_engine = index.as_query_engine()

response = query_engine.query("What are the steps to create a Jira ticket for a production issue?")

display(Markdown(f"{response}"))

Llama.generate: prefix-match hit


 To create a Jira ticket for a production issue, follow these steps:

1. Log in to your Jira account using your credentials.
2. Click on the "Create" button in the top right corner of the screen.
3. Select "Issue" from the dropdown menu.
4. Fill out the required fields for the issue, including:
	* Summary: A brief description of the issue.
	* Description: A detailed explanation of the issue, including its impact and any relevant logs or error messages.
	* Priority: The level of urgency and importance of the issue.
	* Severity: The potential impact of the issue on the system or users.
5. Assign the Jira ticket to a member of the Amazon Development Team responsible for incident resolution.
6. Notify any relevant stakeholders about the new Jira ticket.
7. Submit the Jira issue, resulting in the generation of a unique Jira ticket link.